In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch():
    model.train()
    param_hist = []
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    return torch.tensor(np.mean(param_hist, 0))

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
def make_model():
    return nn.Sequential(
        nn.Conv2d(1, 32, 3),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(32, 64, 3),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(4*4*64, 128),
        nn.ReLU(),
        nn.Linear(128, 10),
        nn.LogSoftmax(-1),
    )

## Epoch

In [7]:
model = make_model()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.0973, validation loss: 2.3022
Epoch 2



Validation accuracy: 0.1725, validation loss: 2.3004
Epoch 3



Validation accuracy: 0.1099, validation loss: 2.2982
Epoch 4


...


Validation accuracy: 0.9753, validation loss: 0.0811
Epoch 27



Validation accuracy: 0.9753, validation loss: 0.0771
Epoch 28



Validation accuracy: 0.9757, validation loss: 0.0790
Epoch 29



Validation accuracy: 0.9775, validation loss: 0.0734
Epoch 30



Validation accuracy: 0.9782, validation loss: 0.0708


In [10]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9838, device='cuda:0'), tensor(0.0556, device='cuda:0'))



Valid: (tensor(0.9782, device='cuda:0'), tensor(0.0708, device='cuda:0'))


In [11]:
optimizer.accelerate()

In [12]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [13]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9838, device='cuda:0'), tensor(0.0539, device='cuda:0'))



Valid: (tensor(0.9778, device='cuda:0'), tensor(0.0710, device='cuda:0'))


## Epoch average

In [14]:
model = make_model()
model.cuda()
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg")

In [15]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.0986, validation loss: 2.3016
Epoch 2



Validation accuracy: 0.1095, validation loss: 2.2993
Epoch 3



Validation accuracy: 0.1095, validation loss: 2.2962
Epoch 4



Validation accuracy: 0.1095, validation loss: 2.2901
Epoch 5


...


Validation accuracy: 0.9782, validation loss: 0.0750
Epoch 27



Validation accuracy: 0.9736, validation loss: 0.0799
Epoch 28



Validation accuracy: 0.9776, validation loss: 0.0743
Epoch 29



Validation accuracy: 0.9790, validation loss: 0.0712
Epoch 30



Validation accuracy: 0.9792, validation loss: 0.0688


In [16]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9845, device='cuda:0'), tensor(0.0525, device='cuda:0'))



Valid: (tensor(0.9792, device='cuda:0'), tensor(0.0688, device='cuda:0'))


In [17]:
optimizer.accelerate()

In [18]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [19]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9756, device='cuda:0'), tensor(0.1015, device='cuda:0'))



Valid: (tensor(0.9725, device='cuda:0'), tensor(0.1106, device='cuda:0'))


## Epoch average, limited to last 20

In [9]:
model = make_model()
model.cuda()
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg", avg_window=20)

In [10]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1579, validation loss: 2.3009
Epoch 2



Validation accuracy: 0.1659, validation loss: 2.2984
Epoch 3



Validation accuracy: 0.1961, validation loss: 2.2949
Epoch 4


...


Validation accuracy: 0.9769, validation loss: 0.0731
Epoch 27



Validation accuracy: 0.9787, validation loss: 0.0688
Epoch 28



Validation accuracy: 0.9777, validation loss: 0.0730
Epoch 29



Validation accuracy: 0.9776, validation loss: 0.0698
Epoch 30



Validation accuracy: 0.9782, validation loss: 0.0691


In [11]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9808, device='cuda:0'), tensor(0.0620, device='cuda:0'))



Valid: (tensor(0.9782, device='cuda:0'), tensor(0.0691, device='cuda:0'))


In [12]:
optimizer.accelerate()

In [13]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [14]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9845, device='cuda:0'), tensor(0.0513, device='cuda:0'))



Valid: (tensor(0.9819, device='cuda:0'), tensor(0.0594, device='cuda:0'))
